# **Restaurants You Will Love**

**CSCE 670: Information Retrieval - Spring 2017** 


## Data Pre-Process
<h5 style="color:red">You don't have to run this file to do the preprocessing. It's very time-cosuming. All generated files are uploaded to Google Drive.</h5>

### Filter out all reviews of restaurants
Yelp challenge dataset contains not only reviews of restaurants but also other bussinesses. The first step is thus from Yelp Challenge data 'yelp_academic_dataset_business.json' to filter out all restaurants bussiness_id. Then get all reviews to restaurants to form 'Restaurants_reviews.json' as raw data for this project.

In [1]:
import json, os
import re

yelp_raw = "yelp_raw_data/yelp_academic_dataset_business.json"
yelp_review = "yelp_raw_data/yelp_academic_dataset_review.json"

In [2]:
All_restaurants = []
raw_business = open(yelp_raw, 'r')
raw_review = open(yelp_review, 'r')

for line in raw_business:
    try: 
        category = json.loads(line)["categories"][0]
    except:
        continue
    if category == 'Restaurants':
        All_restaurants.append(json.loads(line)["business_id"])
    
print All_restaurants.__len__()
output = open("data/Restaurants_review.json",'w')

for line in raw_review:
    try:
        business_id = json.loads(line)["business_id"]
        if business_id in All_restaurants:
             output.write(line)
    except:
        continue
            
output.close()
print "Done!"

Done!


### Refine Data
Collaborative Filter and Lantent Factor both cannot handle 'cold start' problems. The sparsity of input matrix can be regarded as cold start and undermine the training step of our project. So we eliminate new joining retautants which have less than 5 reviews yet, and apply this rule to users as well.

In [3]:
## 1. get the number of users & items in training data
# get the size of user_id & business_id
user_id = []
business_id = []
with open('data/Restaurants_review.json') as f:
    for line in f:
        temp = json.loads(line)        
        user_id.append(temp["user_id"])
        business_id.append(temp["business_id"])
        
# This is without duplication: set
user_id = set(user_id)
business_id = set(business_id)

print "1st part done"

1st part done


In [4]:
## 2. add the "star" into the dictionary
# initialize the dictionary:
#   user_bias_list = {user_id --> a list of scores}
#   business_bias_list = {business_id --> a list of scores}
average = 0
num_star = 0
user_bias_list = {}
business_bias_list = {}
for user_key in user_id:
    user_bias_list[user_key] = []
for business_key in business_id:
    business_bias_list[business_key] = []

# appending the list
with open('data/Restaurants_review.json') as f:
    for line in f:
        # get the infor from file
        temp = json.loads(line)        
        cur_user_id = temp["user_id"]
        cur_business_id = temp["business_id"]
        star = temp["stars"]
        # filling the dictionary & calculate the average
        user_bias_list[cur_user_id].append(star)
        business_bias_list[cur_business_id].append(star)
    
print "2nd part done"

2nd part done


In [5]:
## 3. get the list length of user_bias_list & business_bias_list
beyond_num_user = []
beyond_num_business = []
for key in user_bias_list:
    if user_bias_list[key].__len__() > 5:
        beyond_num_user.append(key)
for key in business_bias_list:
    if business_bias_list[key].__len__() > 5:
        beyond_num_business.append(key)

print "3rb part done"

3rb part done


Because the 'Restaurants_review.json' has over 80,000 records, the next cell is used to break down the large .json file to several files contain only 3,000 records

In [7]:
myfile = open('data/Restaurants_review.json','r')

count = 0
file_number = 0
outfile = open('./temp/final_review' + str(file_number) + '.json','w')
for line in myfile:    
    count += 1
    outfile.write(line)
    if count == 3000:
        count = 0
        file_number += 1
        outfile = open('./temp/final_review' + str(file_number) + '.json','w')
    
print "Done!"

Done!


In [8]:
final_output = open('data/refined_review.json','a+')

i = 0
for infile in os.listdir(os.getcwd() + '/temp'):
    if infile.endswith(".json"):
        infile = open('./temp/'+ infile,'r')
        
        for line in infile:
            user = json.loads(line)["user_id"]
            business = json.loads(line)["business_id"]
    
            if user in beyond_num_user and business in beyond_num_business:
                outfile.write(line)
            
        print "generating " + str(i) + " file"
        i += 1
        
final_output.close()